In [1]:
# STEP 1: Install necessary packages
!pip install transformers==4.40.1 datasets==2.14.4 rouge-score==0.1.2 nltk==3.9.1 --quiet
!pip uninstall -y gcsfs
!pip install --upgrade fsspec==2023.6.0 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 115.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.
Found existing installation: gcsfs 2025.3.2
Uninstalling gcsfs-2025.3.2:
  Successfully uninstalled gcsfs-2025.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires gcsfs!=2025.5.0,>=2023.3.0, wh

In [ ]:
# STEP 2: Restart runtime to apply fixes cleanly
import os
os.kill(os.getpid(), 9)


In [1]:
# STEP 3: Dataset preparation
from datasets import load_dataset

raw_dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")
subset = raw_dataset.select(range(50))  # Select first 50 samples

# Form 5 multi-document samples of 10 docs each
multidoc_test = []
for i in range(0, 50, 10):
    docs = subset[i:i+10]
    articles = docs["article"]
    reference_summary = " ".join(docs["highlights"])
    multidoc_test.append({
        "documents": articles,
        "summary": reference_summary
    })

print("✅ Created", len(multidoc_test), "multi-document samples.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

✅ Created 5 multi-document samples.


In [2]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Load tokenizer and model
dca_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
dca_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
dca_model = dca_model.to(device)

# Define the DCA-style multi-document summary function
def generate_dca_summary(docs, tokenizer, model, device="cpu", max_tokens=512):
    # Each document becomes an "agent" — compress and communicate
    compressed_docs = []
    for doc in docs:
        tokens = tokenizer.tokenize(doc)
        tokens = tokens[:max_tokens]
        compressed = tokenizer.convert_tokens_to_string(tokens)
        compressed_docs.append(compressed)

    # Simulate message exchange: combine all agent outputs
    input_text = " </s> ".join(compressed_docs)
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=1024
    ).to(device)

    with torch.no_grad():
        output_ids = model.generate(
            inputs["input_ids"],
            max_length=150,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [3]:
generated_dca = []

for item in multidoc_test:
    summary = generate_dca_summary(
        item["documents"],
        tokenizer=dca_tokenizer,
        model=dca_model,
        device=device
    )
    generated_dca.append({
        "reference": item["summary"],
        "generated": summary
    })

print("✅ Generated", len(generated_dca), "summaries using DCA-style summarization.")


✅ Generated 5 summaries using DCA-style summarization.


In [4]:
from rouge_score import rouge_scorer
from statistics import mean

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

for pair in generated_dca:
    scores = scorer.score(pair["reference"], pair["generated"])
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rouge2_scores.append(scores["rouge2"].fmeasure)
    rougeL_scores.append(scores["rougeL"].fmeasure)

print("🔍 ROUGE-1:", mean(rouge1_scores))
print("🔍 ROUGE-2:", mean(rouge2_scores))
print("🔍 ROUGE-L:", mean(rougeL_scores))


🔍 ROUGE-1: 0.2579192684863906
🔍 ROUGE-2: 0.08520465989136765
🔍 ROUGE-L: 0.12767457960374862
